In [ ]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
import tensorflow as tf
print(tf.__version__)  # This will print the version of TensorFlow being used.


2.16.1


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input

import json


In [ ]:
DATA_FOLDER = '/content/data/'  # Adjusted path for Colab
POSE_CLASSES = ['chair', 'cobra', 'dog', 'tree', 'warrior']
MODEL_FILENAME = '/content/yoga_pose_model2.h5'  # Adjusted path for Colab


In [ ]:
def load_data(pose_class):
    filepath = f'{DATA_FOLDER}{pose_class}_data.json'  # Direct string concatenation
    with open(filepath, 'r') as file:
        data = json.load(file)
    df = json_to_dataframe(data)

    df['label'] = pose_class
    print(f"Loaded {df.shape[0]} items for class '{pose_class}'")

    return df



In [ ]:
def json_to_dataframe(data):
    rows = []
    # Iterate through each item in the JSON data
    for filename, attributes in data.items():
        # Initialize a dictionary for storing features
        row = {'image': filename}
        # Check if 'features' is a key in your JSON structure
        features = attributes.get('features', {})
        # Iterate through each body part in the features
        for part, feature in features.items():
            # Extract coordinates and score
            coordinates = feature.get('coordinates', {})
            score = feature.get('score', None)
            # Add to the row dictionary
            row[f'{part}_x'] = coordinates.get('x', None)
            row[f'{part}_y'] = coordinates.get('y', None)
            row[f'{part}_score'] = score
        rows.append(row)
    # Convert rows to DataFrame
    return pd.DataFrame(rows)




In [ ]:
def prepare_dataset():
    frames = [load_data(pose_class) for pose_class in POSE_CLASSES]
    full_df = pd.concat(frames, ignore_index=True)

    # Convert labels to categorical
    full_df['label'] = pd.Categorical(full_df['label'])
    full_df['label'] = full_df['label'].cat.codes

    # Split data...
    X = full_df.drop(['label'], axis=1)  # Assuming 'label' is the only non-feature column
    y = full_df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

    return X_train, X_val, X_test, y_train, y_val, y_test


In [ ]:
from tensorflow.keras.optimizers import Adam



def build_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model


In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = prepare_dataset()

# Assuming one-hot encoding for the categorical labels
y_train_encoded = to_categorical(y_train)
y_val_encoded = to_categorical(y_val)

model = build_model(X_train.shape[1], len(POSE_CLASSES))

print(X_train.dtypes)
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_val = X_val.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Check if there are any NaN values after conversion
print(X_train.isna().sum())

# Fill NaN values with the mean (or another statistic) of each column
X_train.fillna(X_train.mean(), inplace=True)
X_val.fillna(X_train.mean(), inplace=True)  # Use training data's statistics for validation set
X_test.fillna(X_train.mean(), inplace=True)  # and for test set

# Recheck dtypes to confirm all are now float64 or a similar numeric type
print(X_train.dtypes)


history = model.fit(X_train, y_train_encoded, epochs=10, validation_data=(X_val, y_val_encoded))

test_loss, test_acc = model.evaluate(X_test, to_categorical(y_test))
print(f'Test accuracy: {test_acc}')

# Save the model to the Colab file system
model.save(MODEL_FILENAME)

Loaded 200 items for class 'chair'
Loaded 200 items for class 'cobra'
Loaded 200 items for class 'dog'
Loaded 200 items for class 'tree'
Loaded 199 items for class 'warrior'
image    object
dtype: object
image    599
dtype: int64
image    float64
dtype: object
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1912 - loss: nan - val_accuracy: 0.2050 - val_loss: nan
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1978 - loss: nan - val_accuracy: 0.2050 - val_loss: nan
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1904 - loss: nan - val_accuracy: 0.2050 - val_loss: nan
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2214 - loss: nan - val_accuracy: 0.2050 - val_loss: nan
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1880 - loss: nan - val_accuracy: 0.2050 - val_loss: nan
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2161 - loss: nan - val_accuracy: 0.2050 - val_loss: nan
Epoch 7/10
19/19 ━

Test accuracy: 0.16500000655651093
